In [0]:
-- ===== 1) ADD CLAIM-LEVEL DENIAL FLAGS =====

USE CATALOG claims360_dev;

-- Claim-level denial signals from 835 detail + map
WITH denial_rollup AS (
  SELECT
    e.claim_id,
    MAX(CASE WHEN m.is_denial THEN 1 ELSE 0 END) AS has_denial_835,
    MAX_BY(m.reason_category, cnt)               AS dominant_denial_category
  FROM (
    SELECT claim_id, reason_code, COUNT(*) AS cnt
    FROM silver.payments_835_events
    WHERE reason_code IS NOT NULL
    GROUP BY claim_id, reason_code
  ) rc
  LEFT JOIN silver.dim_denial_reason_map m
    ON m.code_type = 'CARC' AND m.code = rc.reason_code
  LEFT JOIN silver.payments_835_events e
    ON e.claim_id = rc.claim_id AND e.reason_code = rc.reason_code
  GROUP BY e.claim_id
),
_277_signal AS (
  SELECT
    claim_id,
    MAX(last_277_status_code) AS last_277_status_code
  FROM silver_curated.claims_current
  GROUP BY claim_id
)
CREATE OR REPLACE TABLE gold.fact_claim AS
SELECT
  c.claim_id,
  c.payer_id,
  c.payer_name,
  c.payer_type,
  c.patient_id,
  c.submission_date,
  c.total_charges                                 AS billed_amount,
  COALESCE(c.net_paid_to_date, 0)                 AS net_paid_to_date,
  (c.total_charges - COALESCE(c.net_paid_to_date,0)) AS current_balance,
  c.expected_amount,
  c.last_277_status_code,
  c.last_277_event_ts,
  c.last_835_event_ts,
  c.payment_event_count,
  COALESCE(d.has_denial_835, 0)                   AS has_denial_835,
  CASE
    WHEN COALESCE(d.has_denial_835,0) = 1
         OR COALESCE(c.last_277_status_code,'') IN ('A7','A3','R3')
      THEN 1 ELSE 0
  END                                             AS has_denial_any,
  d.dominant_denial_category
FROM silver_curated.claims_current c
LEFT JOIN denial_rollup d ON d.claim_id = c.claim_id;

In [0]:
-- ===== 2) CREATE FACTS TABLE FOR DENIAL EVENTS =====

CREATE OR REPLACE TABLE gold.fact_denial_event AS
SELECT
  e.claim_id,
  e.payment_ts                AS event_ts,
  e.payer_id,
  e.remit_id,
  e.check_or_eft_trace,
  e.payment_amount,
  e.adjustment_amount,
  e.net_effect,
  e.reason_code,
  COALESCE(m.reason_category, 'Unmapped') AS reason_category
FROM silver.payments_835_events e
LEFT JOIN silver.dim_denial_reason_map m
  ON m.code_type = 'CARC' AND m.code = e.reason_code
WHERE m.is_denial = TRUE;

In [0]:
-- ===== 3) DAILY SNAPSHOT FOR TRENDING / AGING =====

CREATE OR REPLACE TABLE gold.fact_claim_day AS
SELECT
  current_date()                         AS as_of_date,
  c.claim_id,
  c.payer_id,
  c.payer_name,
  c.billed_amount,
  c.net_paid_to_date,
  c.current_balance,
  c.has_denial_any,
  CASE
    WHEN c.current_balance <= 0 THEN 'Closed'
    WHEN DATEDIFF(current_date(), c.submission_date) <= 30  THEN '0–30'
    WHEN DATEDIFF(current_date(), c.submission_date) <= 60  THEN '31–60'
    WHEN DATEDIFF(current_date(), c.submission_date) <= 90  THEN '61–90'
    WHEN DATEDIFF(current_date(), c.submission_date) <= 120 THEN '91–120'
    ELSE '120+'
  END AS aging_bucket
FROM gold.fact_claim c;

In [0]:
-- ===== ACCEPTANCE CHECKS - DELETE LATER =====

-- No duplicates at claim grain
SELECT claim_id, COUNT(*) c FROM gold.fact_claim GROUP BY claim_id HAVING c>1;

-- Reasonable ranges
SELECT
  SUM(CASE WHEN billed_amount < 0 THEN 1 END) AS neg_billed,
  SUM(CASE WHEN net_paid_to_date < 0 THEN 1 END) AS neg_net_paid
FROM gold.fact_claim;

-- Denial mapping coverage
SELECT reason_code, COUNT(*) 
FROM silver.payments_835_events 
WHERE reason_code IS NOT NULL
GROUP BY reason_code
ORDER BY COUNT(*) DESC;